## Resource-Constrained Project Scheduling Problem with Sequence-Dependent Setup Time

This notebook demonstrates how to model and solve the Resource-Constrained Project Scheduling Problem with Sequence-Dependent Setup Times
using Constraint Programming with IBM’s CP Optimizer via the [`docplex.cp`](https://ibmdecisionoptimization.github.io/docplex-doc/cp/refman.html) Python API.
This problem extends the classical RCPSP (see [`rcpsp.ipynb`](https://github.com/radovluk/CP_Cookbook/blob/main/notebooks/rcpsp.ipynb)).

In the RCPSP with setup times, each task has a fixed duration and uses renewable resources with limited capacities, just like in the basic RCPSP. In addition, when two tasks are processed one after the other on the *same* machine, we must also account for a **setup time** between them - such as cleaning, tool changes, or reconfiguration. A feasible schedule must satisfy all precedences, never exceed resource capacities at any moment, and insert the required setup delays between consecutive tasks on each machine. The goal remains to choose start times so that the project finishes as early as possible (minimize the makespan) while respecting **both** the usual RCPSP constraints **and** these setup-time requirements.

### Problem Definition

$$
\begin{aligned}
\min \quad 
& \max_{i \in [1..N]} \mathrm{endOf}(x_i) \qquad 
& \qquad 
& \text{(1)} \\
\text{s.t.} \quad
& \sum_{i \in [1..N]} \mathrm{pulse}(x_i, Q_{ik}) \le C_k, \qquad 
& \forall k \in [1..M] \quad 
& \text{(2)} \\
& \mathrm{endBeforeStart}(x_i, x_j), \qquad 
& \forall (i,j) \in P \quad 
& \text{(3)} \\
& \mathrm{noOverlap}\!\bigl(\mathrm{seq}_r,\ \mathrm{TM}_r\bigr), \qquad
& \forall r \in [1..U] \quad
& \text{(4)} \\
& \text{interval } x_i,\ \text{size} = PT_i, \qquad 
& \forall i \in [1..N] \quad 
& \text{(5)}
\end{aligned}
$$


We are given:

- A finite set of **tasks** indexed by $i \in [1..N]$  
- A finite set of **renewable resources** indexed by $k \in [1..M]$; each resource $k$ has a **capacity** $C_k > 0$.  
- A finite set of **unary (disjunctive) resources** indexed by $r \in [1..U]$ that require **sequence-dependent setup times**. For each $r$, let $I_r \subseteq [1..N]$ be the tasks that run on $r$, and let $\mathrm{TM}_r=[S^{(r)}_{ij}]$ be a **transition (setup) matrix** giving the setup time if task $j$ immediately follows task $i$ on $r$.

Each task $i$:
- requires a **known processing time** $PT_i > 0$,  
- consumes $Q_{ik} \ge 0$ units of each renewable resource $k$ while it runs.

There is a set of **precedence relations** $P \subseteq [1..N]\times[1..N]$,  
where $(i,j)\in P$ means task $i$ must **finish before** task $j$ **starts**.

Objective:
- **(1)** The **makespan** $C_{\max}=\max_i \mathrm{endOf}(x_i)$ — the completion time of the last finishing task — is **minimized**.

Modeling constraints:
- **(2)** At any time, usage on each renewable resource $k$ does not exceed its capacity $C_k$.  
- **(3)** All precedence relations $(i,j)\in P$ are respected.  
- **(4)** For each unary resource $r$, tasks in $I_r$ are processed **without overlap** and with **sequence-dependent setup times** given by $\mathrm{TM}_r$ (i.e., if $j$ follows $i$ on $r$ then $\mathrm{startOf}(x_j)\ge \mathrm{endOf}(x_i)+S^{(r)}_{ij}$, enforced implicitly by $\mathrm{noOverlap}$ with transitions).

Variables:
- **(5)** We decide **start times** for all tasks via interval variables $x_i$ (one per task, fixed size $PT_i$).  
- Additionally, for each unary resource $r$, a **sequence variable** $\mathrm{seq}_r$ orders $\{x_i \mid i\in I_r\}$ and applies the transition matrix $\mathrm{TM}_r$.

#### Symbols and Notation

| Symbol / Function | Meaning | docplex.cp reference |
|---|---|---|
| $N$ | Number of tasks | — |
| $M$ | Number of renewable (cumulative) resources | — |
| $U$ | Number of unary (setup) resources | — |
| $i$ | Task index ($i \in [1..N]$) | — |
| $k$ | Renewable resource index ($k \in [1..M]$) | — |
| $r$ | Unary resource index ($r \in [1..U]$) | — |
| $PT_i$ | Processing time of task $i$ | — |
| $Q_{ik}$ | Renewable resource $k$ usage by task $i$ | — |
| $C_k$ | Capacity of renewable resource $k$ | — |
| $P$ | Precedence set $\subseteq [1..N]\times[1..N]$ | — |
| $x_i$ | Interval variable for task $i$ (size $PT_i$) | [interval_var](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.expression.py.html#docplex.cp.expression.interval_var) |
| $\mathrm{seq}_r$ | Sequence of tasks on unary resource $r$ | [sequence_var](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.expression.py.html#docplex.cp.expression.sequence_var) |
| $\mathrm{TM}_r$ | Transition (setup) matrix on $r$ | [transition_matrix](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.transition_matrix) |
| $\mathrm{pulse}(x_i,Q_{ik})$ | Renewable usage profile of $i$ on $k$ | [pulse](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.pulse) |
| $\mathrm{endBeforeStart}(x_i,x_j)$ | Precedence $i \rightarrow j$ | [end_before_start](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.end_before_start) |
| $\mathrm{noOverlap}(\mathrm{seq}_r,\mathrm{TM}_r)$ | Unary capacity with sequence-dependent setups | [no_overlap](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.no_overlap) |
| $\mathrm{endOf}(x_i)$ | End time of task $i$ | [end_of](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.end_of) |
| $\min \max_i \mathrm{endOf}(x_i)$ | Makespan objective | [minimize](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.minimize) |


### What if we want the setup times to be dependent on renewable non-unary resources?

Turn that renewable resource into several unary resources - if setups are meaningful only when the same copy of the resource is reused,
then you can split the renewable resource into multiple parallel unary resources, one per identical unit.

Keep the resource renewable, but simulate setups with dummy tasks - If the setup represents time when the resource is occupied but no main task runs,
you can create extra tasks that consume the resource during the setup. Create setup intervals that live between consecutive tasks on each machine, and make those setup intervals consume renewable capacity via pulse(...), just like real tasks do.

### Aditional Resources

- **IBM Modeling sequence-dependent setup times**
  - https://www.ibm.com/docs/en/icos/22.1.1?topic=models-modeling-sequence-dependent-setup-times